In [2]:
# ============================================
#   Script : Téléchargement et analyse ADN
#   Auteur : Usto Informatique
#   Objectif : Télécharger une séquence NCBI,
#               la convertir en FASTA et analyser sa composition
# ============================================

from Bio import Entrez, SeqIO
from collections import Counter

# 1️⃣ Configuration Entrez (obligatoire pour accéder à NCBI)
Entrez.email = "alaegmr98@gmail.com"  # Remplace par ton email

# 2️⃣ Télécharger la séquence depuis la base NCBI
accession_id = "PX436992"  # Identifiant d'accès (exemple)

print("🔍 Téléchargement de la séquence depuis NCBI...")
handle = Entrez.efetch(
    db="nucleotide",     # Base de données : séquences ADN/ARN
    id=accession_id,     # Identifiant de la séquence
    rettype="gb",        # Format GenBank
    retmode="text"       # Mode texte lisible
)

# 3️⃣ Lecture du fichier GenBank
record = SeqIO.read(handle, "genbank")
handle.close()
print("✅ Séquence récupérée avec succès.\n")

# 4️⃣ Enregistrement de la séquence au format FASTA
fasta_filename = accession_id + ".fasta"
SeqIO.write(record, fasta_filename, "fasta")
print(f"💾 Fichier FASTA enregistré : {fasta_filename}\n")

# 5️⃣ Affichage des métadonnées
print("=== MÉTADONNÉES ===")
print("Identifiant :", record.id)
print("Description :", record.description)
print("Organisme   :", record.annotations.get("organism", "Inconnu"))

# Récupérer le nom du gène si présent dans les features
gene_name = "Inconnu"
for feature in record.features:
    if feature.type == "gene" and "gene" in feature.qualifiers:
        gene_name = feature.qualifiers["gene"][0]
        break

print("Gène associé:", gene_name)
print("Longueur     :", len(record.seq), "bases\n")

# 6️⃣ Afficher la séquence brute
print("=== SÉQUENCE ===")
print(record.seq)
print("\n")

# 7️⃣ Comptage des bases (A, T, C, G)
sequence = record.seq.upper()
counts = Counter(str(sequence))

print("=== FRÉQUENCES DES BASES ===")
for base in "ATCG":
    print(f"{base} : {counts.get(base, 0)}")

# 8️⃣ Calcul manuel du pourcentage GC
gc_count = counts["G"] + counts["C"]
length = len(sequence)
gc_percent = (gc_count / length) * 100

print(f"\n✅ Pourcentage GC : {gc_percent:.2f}%")


🔍 Téléchargement de la séquence depuis NCBI...
✅ Séquence récupérée avec succès.

💾 Fichier FASTA enregistré : PX436992.fasta

=== MÉTADONNÉES ===
Identifiant : PX436992.1
Description : Homo sapiens isolate AB17 BRCA1 protein (BRCA1) gene, partial cds
Organisme   : Homo sapiens
Gène associé: BRCA1
Longueur     : 350 bases

=== SÉQUENCE ===
CCTGATGGGTTGTGTTTGGTTTCTTTCAGCATGATTTTGAAGTCAGAGGAGATGTGGTCAATGGAAGAAACCACCAAGGTCCAAAGCGAGCAAGAGAATCCCAGGACAGAAAGGTAAAGCTCCCTCCCTCAAGTTGACAAAAATCTCACCCCACCACTCTGTATTCCACTCCCCTTTGCAGAGATGGGCCGCTTCATTTTGTAAGACTTATTACATACATACACAGTGCTAGATACTTTCACACAGGTTCTTTTTTCACTCTTCCATCCCAACCACATAAATAAGTATTGTCTCTACTTTATGAATGATAAAACTAAGAGATTTAGAGAGGCTGTGTATTGGGGATTCCC


=== FRÉQUENCES DES BASES ===
A : 102
T : 100
C : 80
G : 68

✅ Pourcentage GC : 42.29%


In [3]:
# ============================================
# 5️⃣ Requête BLAST en ligne
# ============================================

from Bio.Blast import NCBIWWW, NCBIXML

print("\n🔍 Lancement d'une recherche BLAST en ligne...")
# qblast(programme, base_de_données, séquence)
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

# Sauvegarde du résultat au format XML (optionnel mais conseillé)
blast_xml_file = accession_id + "_blast.xml"
with open(blast_xml_file, "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()
print(f"💾 Résultats BLAST enregistrés dans : {blast_xml_file}\n")

# ============================================
# 6️⃣ Lecture et analyse des résultats BLAST
# ============================================

# Lire le fichier XML
result_handle = open(blast_xml_file)
blast_records = NCBIXML.parse(result_handle)

print("=== MEILLEURS ALIGNEMENTS ===")
for blast_record in blast_records:
    for alignment in blast_record.alignments[:5]:  # Top 5 résultats
        for hsp in alignment.hsps[:1]:  # Premier HSP (High-scoring Pair)
            identity = (hsp.identities / hsp.align_length) * 100
            print(f"Accession : {alignment.accession}")
            print(f"e-value   : {hsp.expect:.2e}")
            print(f"Identité  : {identity:.2f}%\n")
    break  # On ne traite qu’une seule requête à la fois

result_handle.close()
print("✅ Analyse BLAST terminée.")



🔍 Lancement d'une recherche BLAST en ligne...
💾 Résultats BLAST enregistrés dans : PX436992_blast.xml

=== MEILLEURS ALIGNEMENTS ===
Accession : AY273801
e-value   : 2.63e-172
Identité  : 99.14%

Accession : DQ190451
e-value   : 2.63e-172
Identité  : 99.14%

Accession : DQ190455
e-value   : 2.63e-172
Identité  : 99.14%

Accession : L78833
e-value   : 2.63e-172
Identité  : 99.14%

Accession : NG_005905
e-value   : 2.63e-172
Identité  : 99.14%

✅ Analyse BLAST terminée.
